In [8]:
# ULTIMA FORMA
# INTENTANDO MANEJAR LOS ERRORES, con un try except (TENGO QUE METER EL FILTRO DE LOS PAISES Y LA SEMANA)
import time
from pytrends.request import TrendReq
from fastapi import FastAPI
import pandas as pd

app = FastAPI()

def obtener_datos_google_trends_semanal(palabra_clave):
    try:
        pytrends = TrendReq(hl='es', tz=360)
        pytrends.build_payload(kw_list=[palabra_clave], timeframe='2014-01-01 2024-03-31', geo='US')
        data = pytrends.interest_over_time()
        data_semanal = data.resample('W').mean()
        return data_semanal
    except Exception as e:
        print(f"Error al obtener datos de Google Trends: {e}")
        time.sleep(60)
        return obtener_datos_google_trends_semanal(palabra_clave)

@app.get("/google-trends/{topic}")
def google_trends_handler(topic: str):
    trends_data = obtener_datos_google_trends_semanal(topic)  # Usar 'topic' en lugar de 'palabra_clave'
    return trends_data.to_dict()

# Ejemplo de uso
palabra_clave = "Unemployment benefits"
datos = obtener_datos_google_trends_semanal(palabra_clave)
datos


,Unemployment benefits,isPartial
date,,
2014-01-05,25.0,0.0
2014-01-12,NaN,NaN
2014-01-19,NaN,NaN
2014-01-26,NaN,NaN
2014-02-02,17.0,0.0
...,...,...
2024-02-04,5.0,0.0
2024-02-11,NaN,NaN
2024-02-18,NaN,NaN


In [9]:
# PERO LOS QUEREMOS SEMANALES, NO MENSUALES
def obtener_datos_google_trends_semanal(palabra_clave):
    try:
        # Intentar obtener datos de Google Trends
        pytrends = TrendReq(hl='es', tz=360)
        pytrends.build_payload(kw_list=[palabra_clave], timeframe='2014-01-01 2024-03-31', geo='US')
        data = pytrends.interest_over_time(resolution='W')  # ESPECIFICAMOS SEMANAL, pero este parametro no existe!!!
        return data
    except Exception as e:
        # Si ocurre un error, imprimirlo y esperar antes de volver a intentar
        print(f"Error al obtener datos de Google Trends: {e}")
        time.sleep(60)  # Esperar 60 segundos 
        return obtener_datos_google_trends_semanal(palabra_clave)

# Ejemplo de uso
palabra_clave = "Jobs"
datos_semanales = obtener_datos_google_trends_semanal(palabra_clave)
datos_semanales

Error al obtener datos de Google Trends: interest_over_time() got an unexpected keyword argument 'resolution'


KeyboardInterrupt: 

In [2]:
# NO FUNCIONA EL PARAMETRO RESOLUCION para cambiar a semanal, ASI QUE BUSCAMOS OTRA FORMA
import pandas as pd

def obtener_datos_google_trends_semanal(palabra_clave):
    try:
        # Intentar obtener datos de Google Trends
        pytrends = TrendReq(hl='es', tz=360)
        pytrends.build_payload(kw_list=[palabra_clave], timeframe='2014-01-01 2024-03-31', geo='ES')
        data = pytrends.interest_over_time()
        
        # Convertir datos a frecuencia semanal
        data_semanal = data.resample('W').mean()
        
        return data_semanal
    except Exception as e:
        # Si ocurre un error, imprimirlo y esperar antes de volver a intentar
        print(f"Error al obtener datos de Google Trends: {e}")
        time.sleep(60)  # Esperar 60 segundos (ajustar según sea necesario)
        return obtener_datos_google_trends_semanal(palabra_clave)

# Ejemplo de uso
palabra_clave = "Jobs"
datos_semanales = obtener_datos_google_trends_semanal(palabra_clave)
datos_semanales
# NO SALE CORRECTAMENTE PORQUE TIENE LOS DATOS POR MESES, NO POR SEMANA. eL RESAMPLEO NO HA SERVIDO
# SI QUEREMOS PONER DE FILTRO QUE SEA DE 2004 HASTA AHORA, TODO EL HISTORICO, NOS SALE POR MES
# HABRIA QUE DECIDIR SI METER OTROS FILTROS DE TIEMPO (que si que nos de semanales, como los mensuales aunque es mas curro)

,Jobs,isPartial
date,,
2014-01-05,60.0,0.0
2014-01-12,NaN,NaN
2014-01-19,NaN,NaN
2014-01-26,NaN,NaN
2014-02-02,51.0,0.0
...,...,...
2024-02-04,60.0,0.0
2024-02-11,NaN,NaN
2024-02-18,NaN,NaN
